In [1]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Activation, Flatten, BatchNormalization
from keras.utils import np_utils
from keras.optimizers import SGD

import wandb
from wandb.keras import WandbCallback
from keras_contrib.layers import GroupNormalization

from src.freezegroupnorm import FreezeGroupNormalization

Using TensorFlow backend.


In [1]:
MODULE = None
NAME = None
BS = None
GS = None
LR = None

In [ ]:
normalization_module = {
    'batchnorm': BatchNormalization,
    'groupnorm': GroupNormalization,
    'freezegroupnorm': FreezeGroupNormalization 
}

In [ ]:
configs = {
    'module': MODULE,
    'batch_size': BS,
    'group_size': GS,
    'learning_rate': LR
}

wandb.init(
    project='Freeze Group Normalization Exp.',
    name=NAME,
    config=configs
)

config = wandb.config

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
labels=["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]

img_width=28
img_height=28

X_train = X_train.astype('float32')
X_train /= 255.
X_test = X_test.astype('float32')
X_test /= 255.

#reshape input data
X_train = X_train.reshape(X_train.shape[0], img_width, img_height, 1)
X_test = X_test.reshape(X_test.shape[0], img_width, img_height, 1)

# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [ ]:
sgd = SGD(lr=config.learning_rate, nesterov=True)

# build model
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(img_width, img_height,1)))
model.add(normalization_module[config.module]())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (5, 5)))
model.add(normalization_module[config.module]())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=12, batch_size=config.batch_size,
    callbacks=[WandbCallback(data_type="image", labels=labels)])

In [ ]:
model.save(NAME + "_cnn.h5")